### PROYECTO 1: Diseño de un Amplificador Clase A empleando MOSFET




El diseño es un amplificador de potencia de 5 Watt basado en el transistor MRF134 MOSFET. 
La siguiente figura muestra el circuito, que aunque parece completo, es sólo una sugerencia.


![5-cuatrocientos](fig/MRF134_circuito.png)


Se pueden construir un par de amplificadores idénticos que pueden ser conducidos en paralelo, así como tambien rediseñar con varios FETS conectados en paralelo (cuidando las consideraciones de funcionamiento).

El procedimiento para diseñar una clase A es el siguiente.

1. Obtener las impedancias de entrada y salida del FET. 

2. Tensiones de entrada para polarización y exitación. 

3. Encontrar la impedancia óptima de la carga de la línea de carga requerida para entregar la potencia deseada a la carga. 

4. Diseñar redes de adaptación para que coincidan con $Z = 50 \Omega$ en la entrada y la salida. 

El primer paso es obtener la hoja de datos del MRF134. 

http://users.cecs.anu.edu.au/~Gerard.Borg/engn4545_borg/VHFPA/MRF134/mrf134DATA.pdf


La siguiente figura muestra las impedancias del FET.


![5-cuatrocientos](fig/MRF137_Z.png)


En general, las capacitancias están listadas en la hoja de datos, pero no las resistencias.  
Para un FET la resistencia de la fuente de Drain es la única de interés ya que puede ser bastante baja. 
Las capacitancias deben tratarse por separado (utilizando una red de adaptación para eliminar su efecto a alta frecuencia).

La tabla de características eléctricas de la hoja de datos da la capacitancia de Gate a Source, $C_{GS} = 20-50 pF$ y la capacitancia de Drain a fuente $C_{DS} = 11-25 pF$. 
Estos tienen una incertidumbre bastante grande que deberá tenerse en cuenta al hacer el diseño. 

La capacitancia de Drain a la puerta $C_{DG} = 4-5 pF$ y no es despreciable, especialmente porque será amplificada por el efecto Miller. 
También podría conducir a la inestabilidad y podría tener que ser neutralizada.

La siguiente figura muestra las características corriente tensión del FET. 

![5-cuatrocientos](fig/CLASSA_3.png)

![image.png](fig/MRF137_IV.png)

Estas son las características de transferencia y salida del FET tomadas de la hoja de datos. 

Obsérve que la tensión Gate a Source (VGS) es lo que impulsa al FET. 
Esto produce excursiones en la corriente de Drain a Source (IDS) que a su vez produce la oscilación del voltaje de salida al variar el voltaje de Drain a Source(VDS).


Asumimos que la resistencia de carga que requerimos es la que da nuestra potencia deseada para la máxima excursión posible del voltaje de salida, se calcula como $R_{LOAD} = \frac{(V_{DD} - V_{sat})^2}{2P_L}$, donde $V_{sat}$ es el valor mínimo que la tensión de la fuente de Drain puede tener. 

Lo primero que hay que entender del clase A es que vamos operar al transistor de tal manera que la salida produzca la excursión completa de una onda sinusoidal lo mejor que pueda para la potencia dada.

El siguiente circuito muestra la excursión de la salida para la máxima transferencia de potencia asumiendo que Vsat = 0. 

También usaremos esta figura para discutir los modos no lineales de alta eficiencia (clase AB,B y C) dentro de poco. Utilizamos una reactancia (resistencia de CC igual a cero) para conectar el desagüe a la fuente de alimentación, de modo que el sesgo de CC del desagüe sea igual al voltaje de la fuente (VCC). 
La transferencia óptima de energía ocurrirá cuando la resistencia de carga (que se ve aquí absorbiendo CDS en la red de adaptación de salida) tenga el valor correcto para hacer que la excursión de la corriente máxima sea igual a (VCC -Vsat)/Rload. No es posible que se produzcan oscilaciones de mayor amplitud que ésta, ya que bajo ninguna circunstancia un FET (o un BJT) con una alimentación entre tierra y un valor positivo puede conducir nunca una corriente negativa.



No olvides que estamos asumiendo que la red de adaptación ya está funcionando para producir la impedancia de carga. 

Recuerden que no podemos hacer este diseño para 1 vatio para un solo amplificador porque en la clase A, la CARGA y el FET disipan la misma potencia y 1 Watt excede EXCEDE la potencia nominal del 2N7000.

El voltaje de CC en el Drain, VDS (bajo ninguna excitación) debe ser VCC. El valor mínimo de VDS es Vsat. La excursión máxima de pico en el desagüe es VCC + VCC - Vsat = 2VCC - Vsat y la oscilación de pico a pico es 2(VCC - Vsat). La corriente máxima en la carga es ILOAD = (VCC - Vsat)/CARGA y esto debe lograrse cuando la corriente de Drain es cero Id = 0.


El voltaje a través de la carga, Vt en cualquier momento es dado por,

$Vt = (VCC - Vsat)/CARGA sin(Wt)$

La corriente a través de la carga, en cualquier momento es dada por,
It = Vt / RLOAD

Debido a que el condensador de bloqueo tiene un voltaje de reposo VCC a través de él, el voltaje de Drain es dado por,
VDS = VCC + Vt

La corriente en el estrangulador de la fuente de alimentación es una constante y viene dada por la corriente de carga cuando el FET se corta de la siguiente manera,
ICHOKE = (VCC - Vsat) / RLOAD

En el medio ciclo negativo de Vt la corriente de Drain se obtiene del condensador de desacoplamiento que actúa como una especie de fuente de alimentación en esta circunstancia con una corriente igual del estrangulador y es dada por,
max(IDS) = 2*(VCC - Vsat) / CARGA

El valor mínimo del IDS se produce cuando el voltaje de Drain alcanza su máximo y es,
min(IDS) = 0

Así que el IDS está dado por
IDS = (VCC - Vsat)/RLOAD - Vt/RLOAD;

Finalmente podemos escribir la línea de carga que es la relación entre IDS y VDS,
IDS = (2VCC - Vsat - VDS)/CARGA;

Algún código de MATLAB que hace esto por ti se adjunta al final de esta página.

Si queremos 0,25 vatios de potencia para un suministro de 5 voltios, entonces usando la fórmula anterior y estimando Vsat = 0,2 voltios de las características IV anteriores, obtenemos la siguiente línea de carga,


Observe que esto produce un valor conveniente accidental de CARGA = 46 Ohm. Como esto puede ser también 50 Ohms tenemos una tarea trivial de adaptación y podemos usar cualquier viejo filtro de paso bajo de Chebyshev para adaptarlo al preselector de antena (50 Ohms). 
Tenga en cuenta que todavía necesita una red de adaptación porque todavía tiene que absorber los 25 pF de capacitancia del CDS.

Para hacer coincidir la entrada, sólo tiene que utilizar las técnicas de adaptación para hacer coincidir la señal de 20 dBm de una fuente de 50 Ohms con la impedancia de entrada. Note que la impedancia de entrada del FET es sólo CGS = 50 pF en paralelo con R1 = 68 Ohms en la figura simplificada de abajo. El valor real de R1 está determinado enteramente por el sesgo para poner la FET en el régimen de clase A de las características IV anteriores. Debido a que el CGS es tan grande (-j80 Ohms a 40 MHz), la parte real de la combinación paralela de CGS y R1 se divide de cualquier manera.
